<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -qU langchain-pinecone pinecone-notebooks requests langchain-groq langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00


## Configuração de Variaveis

In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

### Geração de embeddings e indíces do banco vetorial

In [3]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [4]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [5]:
import time

index_name = "index-ppc-markdown-dev"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

## Vector store por Inference API do Hugging Face (lento)

In [6]:
# import requests
# from langchain_pinecone import PineconeVectorStore

# class CustomEmbedding:
#     def __init__(self):
#         self.hf_token = settings.HF_TOKEN
#         self.model_id = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
#         self.api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{self.model_id}"
#         self.headers = {"Authorization": f"Bearer {self.hf_token}"}

#     def embed_query(self, query):
#         return self.get_embeddings([query])[0]

#     def embed_documents(self, documents):
#         return self.get_embeddings(documents)

#     def get_embeddings(self, texts):
#         response = requests.post(self.api_url, headers=self.headers, json={"inputs": texts, "options": {"wait_for_model": True}})
#         response.raise_for_status()  # Raise an error for bad responses
#         return response.json()

# def initialize_vector_store(index):
#     # Initialize the custom embedding class
#     custom_embedding = CustomEmbedding()

#     # Initialize PineconeVectorStore with the custom embedding class
#     vector_store = PineconeVectorStore(index=index, embedding=custom_embedding)

#     return vector_store


In [7]:
# vector_store = initialize_vector_store(index)
# results = vector_store.similarity_search("Limite de Atividades de ensino", k=2)

## Vector store local

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [11]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## Geração da resposta

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)



def query_transformation(query):

    query_transformation_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""Esse é minha pergunta: {query}
    Me retorne uma string que capture os itens mais essenciais da pergunta.
    Caso seja uma pergunta, transforme em algo mais conciso. Exemplo:
    "Qual a ementa de Teoria da Computação?" --> "TEORIA DA COMPUTAÇÃO | EMENTA".
    Retorne apenas a string sem nenhuma frase introdutória.
    """

    # Criando o prompt para a transformação da consulta
    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    # Gerando a consulta transformada
    chain = prompt_template | query_transformation_llm
    response = chain.invoke({})
    return response.content


def get_context(query, source_name = 'ppc2024.md'):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    return  '@@@@\n\n\n'.join([result.page_content for result in results])




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua tarefa é fornecer respostas precisas sobre qualquer questão relacionada ao PPC, utilizando o contexto fornecido para guiar sua resposta."
                "Utilize apenas o contexto informado como referencia"
                "Caso a resposta não está no contexto, informe que não sabe a resposta e recomende consultar o documento original."

                "Responda exclusivamente em portugues brasileiro, em formato markdown"
            ),
            (
            "human",
            f"Dado o contexto: {contexto}\n\nPergunta: {pergunta}. Utilize apenas o contexto fornecido para responder"),
        ]
    )
    return prompt

def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    return response


def get_answer(query, source_name):
    expanded_query = query_transformation(query) + query
    print('Query expandida: ', expanded_query)

    context = get_context(query, source_name)
    prompt = build_prompt(context, query)
    response = get_completion(prompt)


    return context, response.content, expanded_query


## Perguntas

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [18]:
!mkdir -p data
!cp /content/drive/My\ Drive/tcc/data/data/* data/


In [37]:
def gerar_perguntas(ppc):
    # Disciplinas do PPC 2018
    disciplinas_ppc_2018 = [
        "Informática Básica",
        "Circuitos Digitais",
        "Sistemas Digitais",
        "Pesquisa e Ordenação de Dados",
        "Programação I",
        "Programação II",
        "Engenharia de Software I",
        "Engenharia de Software II",
        "Computação Gráfica",
        "Redes de Computadores",
        "Planejamento e Gestão de Projetos",
        "Construção de Compiladores",
        "Algoritmos e Programação",
        "Estatística Básica",
        "Cálculo I",
        "Cálculo II",
        "Matemática Discreta",
        "Álgebra Linear",
        "Banco de Dados I",
        "Banco de Dados II",
        "Linguagens Formais e Autômatos",
        "Sistemas Operacionais",
        "Linguagens de Programação",
        "Inteligência Artificial",
        "Teoria da Computação",
        "Grafos",
        "Teoria da Computação",
        "Cálculo Numérico",
        "Trabalho de Conclusão de Curso I",
        "Trabalho de Conclusão de Curso II"
    ]

    # Disciplinas do PPC 2024
    disciplinas_ppc_2024 = [
        "Desenvolvimento de Sistemas Web",
        "Introdução à Computação",
        "Banco de Dados I",
        "Matemática C",
        "Estatística Básica",
        "Produção Textual Acadêmica",
        "História da Fronteira Sul",
        "Meio Ambiente, Economia e Sociedade",
        "Direitos e Cidadania",
        "Introdução à Filosofia",
        "Geometria Analítica",
        "Álgebra Linear",
        "Probabilidade e Estatística",
        "Cálculo I",
        "Cálculo II",
        "Matemática Discreta",
        "Cálculo Numérico",
        "Análise de Algoritmos",
        "Modelagem e Simulação",
        "Computação Distribuída",
        "Empreendedorismo",
        "Segurança e Auditoria de Sistemas",
        "Gestão da Inovação",
        "Empreendedorismo e Criação de Negócios",
        "Sistemas Operacionais",
        "Linguagens de Programação",
        "Inteligência Artificial",
        "Teoria da Computação",
        "Grafos",
        "Banco de Dados II",
        "Linguagens Formais e Autômatos",
        "Planejamento e Gestão de Projetos",
        "Trabalho de Conclusão de Curso I",
        "Trabalho de Conclusão de Curso II"
    ]

    # Identificar as disciplinas exclusivas para cada PPC
    if ppc == '2018':
        disciplinas_especificas = list(set(disciplinas_ppc_2018) - set(disciplinas_ppc_2024))
    elif ppc == '2024':
        disciplinas_especificas = list(set(disciplinas_ppc_2024) - set(disciplinas_ppc_2018))
    else:
        return "PPC inválido! Escolha '2018' ou '2024'."

    perguntas = []
    for disciplina in disciplinas_especificas:
        perguntas.append(f"Qual é a ementa de {disciplina}?")

    # Perguntas comuns divididas por categorias
    perguntas_comuns = [
        # Perguntas sobre carga horária
        "Qual o limite de horas para atividades de extensão?",
        "Qual o limite de horas para atividades de cultura?",
        "Qual o limite de horas para atividades complementares?",
        "Qual o limite de horas para atividades de ensino?",
        "Quantas horas de estágio são obrigatórias no curso?",
        "Quantas horas de ACC preciso para me formar?"

        # Perguntas sobre objetivos e perfil do curso
        "Qual é o objetivo geral do curso de Ciência da Computação?",
        "Quais são os objetivos específicos do curso?",
        "Como é o perfil do egresso do curso de Ciência da Computação?",

        # Perguntas sobre a estrutura curricular
        "Qual é a carga horária total do curso?",
        "Qual a carga horária das Atividades Complementares?",
        "Quais disciplinas fazem parte do Domínio Comum?",
        "Quais são os componentes curriculares do Domínio Específico?",
        "Qual é a grade curricular do vespertino?",
        "Qual é a grade curricular do noturno?",
        "Quais disciplinas compõem o Domínio Comum do curso?",
        "Quais disciplinas fazem parte do Domínio Conexo?",
        "Quais são os componentes curriculares do Domínio Específico?",
        "Como é distribuída a carga horária entre teoria e prática?",
        "Há possibilidade de realizar disciplinas EAD?",

        # Perguntas sobre metodologia e avaliação
        "Como é o processo de avaliação do ensino e da aprendizagem?",
        "Como o TCC I é avaliado?",
        "Como posso validar o TCC?",
        "Como funciona o TCC Startup?",
        "Como posso validar meu artigo científico como TCC?"

        # Perguntas sobre histórico do curso e da universidade
        "Qual é o histórico da Universidade Federal da Fronteira Sul?",
        "Quando foi criado o curso de Ciência da Computação?",

        # Perguntas sobre infraestrutura
        "Quais laboratórios estão disponíveis para os alunos do curso?",

        # Perguntas sobre extensão
        "Como o curso promove a integração entre ensino, pesquisa e extensão?",
        "Quais são as atividades de extensão relacionadas ao curso?",

        # Perguntas sobre mudanças no PPC
        "Por que mudou o PPC?",

        # Perguntas sobre processo seletivo e ingresso
        "Como é realizado o processo seletivo para ingresso no curso de Ciência da Computação?",
        "Quais são as formas de ingresso disponíveis para novos alunos?",
        "O curso oferece vagas para transferência externa ou interna?"
    ]

    perguntas.extend(perguntas_comuns)

    return perguntas

# Exemplo de uso
perguntas_2018 = gerar_perguntas('2018')
perguntas_2024 = gerar_perguntas('2024')



In [39]:
len(perguntas_2018), len(perguntas_2024)

(46, 51)

In [51]:
import csv
import time


with open('QA_ppc2018.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Pergunta', 'Resposta', 'Query Expandida'])  # Cabeçalhos

    for question in perguntas_2018:
        context, response, expanded_query = get_answer(question, 'ppc2018')
        writer.writerow([question, response, expanded_query])
        print(f"Pergunta: {question}")
        print(f"Resposta: {response}")
        time.sleep(7)

Query expandida:  CONSTRUÇÃO DE COMPILADORES | EMENTA 
Qual é a ementa de Construção de Compiladores?
Pergunta: Qual é a ementa de Construção de Compiladores?
Resposta: A ementa do componente curricular "Construção de Compiladores" (GEX108) é:

- Projeto de especificação de linguagens de programação.
- Etapas que compreendem o processo de compilação: análise léxica, análise sintática, análise semântica, geração e otimização de código.
- Evolução e tendências da área de compiladores e linguagens de programação.
- Implementação de analisadores.
Query expandida:  CIRCUITOS DIGITAIS | EMENTA 
Qual é a ementa de Circuitos Digitais?
Pergunta: Qual é a ementa de Circuitos Digitais?
Resposta: A ementa de Circuitos Digitais é:

Fundamentos de Eletrônica. Famílias Lógicas. Simulação de Circuitos Lógicos. Álgebra de Boole. Circuitos Combinacionais: codificadores, multiplexadores e aritméticos. Circuitos Sequenciais: Latches, Flip-Flops e Registradores. Memórias.

Essa informação pode ser encontra

In [57]:
import csv
import time


with open('QA_ppc2024.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Pergunta', 'Resposta', 'Query Expandida'])  # Cabeçalhos

    for question in perguntas_2024:
        context, response, expanded_query = get_answer(question, 'ppc2024')
        writer.writerow([question, response, expanded_query])
        print(f"Pergunta: {question}")
        print(f"Resposta: {response}")
        time.sleep(5)

Query expandida:  EMENTA | MEIO AMBIENTE, ECONOMIA E SOCIEDADE 
Qual é a ementa de Meio Ambiente, Economia e Sociedade?
Pergunta: Qual é a ementa de Meio Ambiente, Economia e Sociedade?
Resposta: A ementa do componente curricular Meio Ambiente, Economia e Sociedade é: "Modos de produção: organização social, Estado, mundo do trabalho, ciência e tecnologia. Elementos de economia ecológica e política. Estado atual do capitalismo. Modelos produtivos e sustentabilidade. Experiências produtivas alternativas." Essa informação está localizada no Código do componente curricular, na coluna "EMENTA".
Query expandida:  EMENTA DE ANÁLISE DE ALGORITMOS 
Qual é a ementa de Análise de Algoritmos?
Pergunta: Qual é a ementa de Análise de Algoritmos?
Resposta: A ementa do componente curricular Análise de Algoritmos é:

- Prova de corretude e de parada de algoritmos.
- Assintótica: notações O, o, Ω, ω e Θ.
- Análise de complexidade de tempo e de espaço: melhor caso, pior caso e caso-médio.
- Recorrências.

In [58]:
import shutil
from datetime import datetime
import pytz

def save_csv_to_drive(csv_file_path, drive_folder_path):
    # Define the timezone for BRT
    brt_timezone = pytz.timezone('America/Sao_Paulo')

    file_name = csv_file_path.split('.csv')[0]

    # Generate a timestamp in BRT timezone
    timestamp = datetime.now(brt_timezone).strftime('%d_%m_%Y_%H-%M-%S')

    # Define the path in Google Drive with timestamp
    drive_path = f'{drive_folder_path}/{csv_file_path}_{timestamp}.csv'

    # Copy the CSV file to Google Drive
    shutil.copy(csv_file_path, drive_path)
    print(f'File saved to: {drive_path}')



In [56]:
csv_file_path = 'QA_ppc2018.csv'
drive_folder_path = '/content/drive/MyDrive/tcc'
save_csv_to_drive(csv_file_path, drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/QA_ppc2018.csv_29_08_2024_23-42-22.csv


In [59]:
save_csv_to_drive('QA_ppc2024.csv', drive_folder_path)

File saved to: /content/drive/MyDrive/tcc/QA_ppc2024.csv_30_08_2024_00-06-21.csv
